In [11]:
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from dotenv import load_dotenv
load_dotenv()

True

In [12]:
azure_api_key=os.getenv("AZURE_OPENAI_API_KEY")
api_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_version=os.getenv("OPENAI_API_VERSION")
open_ai_api_key=os.getenv("OPENAI_API_KEY")

In [13]:
embedding_model = AzureOpenAIEmbeddings(azure_deployment="text-embedding")
llm = AzureChatOpenAI(azure_deployment="gpt-35-turbo-1106", temperature=0)

# Load document

In [4]:
from langchain_community.document_loaders import PyPDFLoader
# for pdf to LaTeX
# loader = MathpixPDFLoader("data/luri_higher_topos.pdf")
# data = loader.load()

loader = PyPDFLoader("data/luri_higher_topos.pdf")
data = loader.load()


# Split text

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200B",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
)

splits = text_splitter.split_documents(data)

In [10]:
# Split semantically based on embeddings
# from langchain_experimental.text_splitter import SemanticChunker

# semantic_splitter = SemanticChunker(AzureOpenAIEmbeddings(azure_deployment="text-embedding"))
# semantic_splits = semantic_splitter.create_documents("insert page content only")

In [6]:
# vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory="db/recursive_splits")
# load vectorstore
vectorstore = Chroma(persist_directory="db/recursive_splits", embedding_function=embedding_model)

# Retrieval

In [7]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":10})

results = retriever.get_relevant_documents("Can you give some basic examples of a Kan complex?")

In [8]:
results

[Document(page_content='In the setting of simplicial sets, this problem admits an att ractive formulation in terms of Quillen’s theory\nofminimal Kan complexes. Every Kan complex Xis homotopy equivalent to a minimal Kan complex, and\na mapX→Yof minimal Kan complexes is a homotopy equivalence if and onl y if it is an isomorphism.\nConsequently, the classiﬁcation of Kan complexes up to homo topy equivalence is equivalent to the classi-\nﬁcation of minimal Kan complexes up to isomorphism. Of course, in practical ter ms, this is not of much\nuse for solving the classiﬁcation problem. Nevertheless, t he theory of minimal Kan complexes (and, more\ngenerally, minimal Kan ﬁbrations) is a useful tool in the hom otopy theory of simplicial sets. The purpose\nof this section is to describe a generalization of the theory of minimal models, in which Kan ﬁbrations are\nreplaced by inner ﬁbrations. An exposition of this theory ca n also be found in [44].', metadata={'page': 90, 'source': 'data/luri_hi

# Generation

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful math research assistant."),
        ("human", """Use the following pieces of retrieved context to answer the question. 
        If you don't know the answer, just say that you don't know.
        Question: {question}
        Context: {context}
        Answer:""")
    ]
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
rag_chain.invoke("Why are smooth functions on smooth spaces a Kan complex?")

TypeError: Completions.create() got an unexpected keyword argument 'azure_api_version'